In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/covid-toy/covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [5]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='has_covid'),
                                                    df['has_covid'],
                                                    test_size=0.2,
                                                    random_state=0)

In [10]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


# **Adding simple imputer to fever col**

In [11]:
si=SimpleImputer()

In [12]:
X_train_fever=si.fit_transform(X_train[['fever']])

In [13]:
X_test_fever=si.fit_transform(X_test[['fever']])

In [14]:
X_train_fever

array([[ 99.        ],
       [104.        ],
       [ 98.        ],
       [104.        ],
       [ 99.        ],
       [ 99.        ],
       [101.        ],
       [ 98.        ],
       [ 98.        ],
       [100.        ],
       [102.        ],
       [ 99.        ],
       [ 99.        ],
       [104.        ],
       [100.        ],
       [102.        ],
       [ 98.        ],
       [100.91549296],
       [100.        ],
       [104.        ],
       [100.        ],
       [ 98.        ],
       [100.        ],
       [100.91549296],
       [101.        ],
       [103.        ],
       [ 98.        ],
       [102.        ],
       [101.        ],
       [100.91549296],
       [100.91549296],
       [104.        ],
       [103.        ],
       [102.        ],
       [104.        ],
       [101.        ],
       [ 98.        ],
       [102.        ],
       [ 98.        ],
       [100.        ],
       [100.91549296],
       [103.        ],
       [104.        ],
       [ 99

In [15]:
X_train_fever.shape

(80, 1)

# **Ordinalencoding -> cough**

In [16]:
custom_categories=catagories=[['Mild','Strong']]
oe=OrdinalEncoder(categories=custom_categories)
X_train_cough=oe.fit_transform(X_train[['cough']])

X_test_cough=oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

# **OneHotEncoding -> gender,city**

In [17]:
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [18]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [19]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

In [20]:
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

In [21]:
X_train_transformed.shape

(80, 7)

# **Using Sklearn Column Transformer**

In [22]:
from sklearn.compose import ColumnTransformer

In [23]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=custom_categories),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [24]:
transformer.fit_transform(X_train).shape

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [25]:
transformer.fit_transform(X_test).shape

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)